In [91]:
#import all packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score

# 1. Comment Data

In [237]:
data_comment = pd.read_csv('./data/commentTestData.csv', error_bad_lines=False, sep='\t')

In [238]:
data_comment.head()

,Id,PostId,Score,Text,CreationDate,UserId,link,outdated (manually checked),description,link outdated,...,phrases,strucutures,point out deprecated version,provide alternatives,share a link,provide code,others,other notes,link to updated post,for question (1) / for answer (2)
0,29746988,5502390,2,The test page http://blog.kosny.com/testpages/...,2013-11-14T15:46:55.083,595094,https://stackoverflow.com/questions/5502390,True,code format in answer outdated,0.0,...,NaN,NaN,1.0,1.0,1.0,0.0,0.0,NaN,NaN,2.0
1,22037703,15550433,0,@believeme That example uses deprecated mysql ...,2013-03-21T15:32:21.667,790224,https://stackoverflow.com/questions/15550433,True,another comment outdated,0.0,...,NaN,NaN,1.0,0.0,0.0,0.0,0.0,outdated mysql library,NaN,1.0
2,58927510,33262467,1,"This is the best answer now, as aws 1 is depre...",2016-02-25T09:05:49.150,993592,https://stackoverflow.com/questions/33262467,True,comment in one answer to point out another ans...,0.0,...,NaN,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,2.0
3,102026242,42477724,1,"Is this still meant to work, because it appear...",2019-09-04T23:18:00.197,3674263,https://stackoverflow.com/questions/42477724,False,asks for possible outdated version,0.0,...,possibly outdated,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,2.0
4,111893971,63275335,0,Hi. I tried /home/webapp as well and that didn...,2020-08-06T06:06:30.027,1304505,https://stackoverflow.com/questions/63275335,False,asks for possible outdated version,0.0,...,seems to be outdated,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,2.0


In [239]:
data_comment.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId', 'link',
       'outdated (manually checked)', 'description', 'link outdated',
       'answer outdated', 'question outdated', 'already outdated when posted',
       'other reasons', 'uncertain', 'partly outdated', 'totally outdated',
       'is aws-related', 'related service', 'keywords', 'phrases',
       'strucutures', 'point out deprecated version', 'provide alternatives',
       'share a link', 'provide code', 'others', 'other notes',
       'link to updated post', 'for question (1) / for answer (2)'],
      dtype='object')

In [242]:
#drop unuseful features
data_comment = data_comment.drop(columns=['description', 'uncertain', 'phrases', 'strucutures', 
                                          'other notes', 'link to updated post'])

In [243]:
data_comment.head()

,Id,PostId,Score,Text,CreationDate,UserId,link,outdated (manually checked),link outdated,answer outdated,...,totally outdated,is aws-related,related service,keywords,point out deprecated version,provide alternatives,share a link,provide code,others,for question (1) / for answer (2)
0,29746988,5502390,2,The test page http://blog.kosny.com/testpages/...,2013-11-14T15:46:55.083,595094,https://stackoverflow.com/questions/5502390,True,0.0,0.0,...,1.0,no,NaN,out of date,1.0,1.0,1.0,0.0,0.0,2.0
1,22037703,15550433,0,@believeme That example uses deprecated mysql ...,2013-03-21T15:32:21.667,790224,https://stackoverflow.com/questions/15550433,True,0.0,0.0,...,0.0,no,NaN,deprecated,1.0,0.0,0.0,0.0,0.0,1.0
2,58927510,33262467,1,"This is the best answer now, as aws 1 is depre...",2016-02-25T09:05:49.150,993592,https://stackoverflow.com/questions/33262467,True,0.0,1.0,...,0.0,yes,aws-sdk-ruby,deprecated,1.0,0.0,0.0,0.0,0.0,2.0
3,102026242,42477724,1,"Is this still meant to work, because it appear...",2019-09-04T23:18:00.197,3674263,https://stackoverflow.com/questions/42477724,False,0.0,0.0,...,0.0,yes,amazon-elastic-beanstalk,outdated,1.0,0.0,0.0,0.0,0.0,2.0
4,111893971,63275335,0,Hi. I tried /home/webapp as well and that didn...,2020-08-06T06:06:30.027,1304505,https://stackoverflow.com/questions/63275335,False,0.0,0.0,...,0.0,yes,"amazon-ec2, amazon-elastic-beanstalk",outdated,1.0,0.0,0.0,0.0,0.0,2.0


## 1.1 BoW

In [282]:
#use only text as training data
X = data_comment['Text']
y = data_comment['outdated (manually checked)']

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [284]:
len(X_train)

126

In [296]:
X_train = X_train.fillna('fillna')

In [297]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [12]:
X_train_counts.shape

(126, 1268)

In [298]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [289]:
X_train_tfidf.shape

(126, 1324)

In [299]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)
clfLR = LogisticRegression(C=50).fit(X_train_tfidf, y_train)

In [300]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [301]:
predicted = clf.predict(X_test_tfidf)

In [302]:
clf.score(X_test_tfidf, y_test)

0.71875

In [303]:
clfLR.score(X_test_tfidf, y_test)

0.75

In [304]:
f1_score(y_test, clfLR.predict(X_test_tfidf))

0.8518518518518519

In [305]:
f1_score(y_test, clf.predict(X_test_tfidf))

0.8363636363636363

## 1.2 Other Features

In [115]:
X = data_comment[['Score', 'link outdated',
       'answer outdated', 'question outdated', 
       'other reasons', 'partly outdated', 'totally outdated',
       'is aws-related', 'keywords',
       'point out deprecated version', 'provide alternatives',
       'share a link', 'provide code', 'others',
       'for question (1) / for answer (2)']]
y = data_comment['outdated (manually checked)']

In [116]:
X.fillna('0')
key_word_dummies = pd.get_dummies(X.keywords, prefix='keyword_')
aws_related_dummies = pd.get_dummies(X['is aws-related'], prefix='aws_related_')
X = pd.concat([X, key_word_dummies], axis = 1)
X = pd.concat([X, aws_related_dummies], axis = 1)
X = X.drop(columns = 'keywords')
X = X.drop(columns = 'is aws-related')

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [118]:
X_train = X_train.fillna('0')
X_test = X_test.fillna('0')

In [24]:
len(X_train)

126

In [119]:
clf = BernoulliNB().fit(X_train, y_train)
clfLR = LogisticRegression(C=50).fit(X_train, y_train)

In [120]:
clf.score(X_test, y_test)

0.78125

In [121]:
clfLR.score(X_test, y_test)

0.96875

In [27]:
f1_score(y_test, clf.predict(X_test))

0.8571428571428571

In [122]:
f1_score(y_test, clfLR.predict(X_test))

0.9777777777777777

In [28]:
clf.predict(X_test)

array([False,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False, False, False, False,  True,  True, False, False,  True,
       False, False,  True,  True, False])

In [29]:
y_test

48     False
65      True
105    False
49      True
66     False
152    False
153     True
76      True
132     True
135     True
50     False
54      True
30      True
147     True
45     False
57      True
9       True
34      True
0       True
46      True
83     False
148    False
2       True
104     True
44     False
3      False
92      True
70     False
74     False
56      True
16     False
40     False
Name: outdated (manually checked), dtype: bool

# 2. Answer Data

In [30]:
data_answer = pd.read_csv('./data/AnswerTestData_1.csv', encoding="ISO-8859-1")

## 2.1 BoW

In [332]:
#use only text as training data
X = data_answer['Text']
y = data_answer['outdated (manually checked)']

In [333]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [334]:
X_train = X_train.fillna('fillna')
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [335]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)
clfLR = LogisticRegression(C=50).fit(X_train_tfidf, y_train)

In [336]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [337]:
clf.score(X_test_tfidf, y_test)

0.75

In [338]:
clfLR.score(X_test_tfidf, y_test)

0.8

In [339]:
f1_score(y_test, clf.predict(X_test_tfidf))

0.8571428571428571

In [340]:
f1_score(y_test, clfLR.predict(X_test_tfidf))

0.8749999999999999

## 2.2 Other Features

In [38]:
data_answer.columns

Index(['Unnamed: 0', 'Id', 'PostId', 'link', 'outdated (manually checked)',
       'description', 'link outdated', 'answer outdated',
       'service in question outdated', 'other reasons', 'partly outdated',
       'totally outdated', 'is aws-related', 'related service', 'keywords',
       'phrases', 'strucutures', 'point out deprecated version',
       'provide alternatives', 'share a link', 'provide code', 'others',
       'Text'],
      dtype='object')

In [39]:
X = data_answer[['link outdated',
       'answer outdated', 'service in question outdated', 'other reasons',
       'partly outdated', 'totally outdated', 'is aws-related',
       'keywords', 
       'point out deprecated version', 'provide alternatives', 'share a link',
       'provide code', 'others']]
y = data_answer['outdated (manually checked)']

In [40]:
X.fillna('0')
key_word_dummies = pd.get_dummies(X.keywords, prefix='keyword_')
aws_related_dummies = pd.get_dummies(X['is aws-related'], prefix='aws_related_')
X = pd.concat([X, key_word_dummies], axis = 1)
X = pd.concat([X, aws_related_dummies], axis = 1)
X = X.drop(columns = 'keywords')
X = X.drop(columns = 'is aws-related')

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
X_train = X_train.fillna('0')
X_test = X_test.fillna('0')

In [43]:
clf = BernoulliNB().fit(X_train, y_train)

In [44]:
clf.score(X_test, y_test)

0.85

In [45]:
f1_score(y_test, clf.predict(X_test))

0.896551724137931

# 3. Combined Data (Comment&Answer)

In [46]:
data_comment.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId', 'link',
       'outdated (manually checked)', 'link outdated', 'answer outdated',
       'question outdated', 'already outdated when posted', 'other reasons',
       'partly outdated', 'totally outdated', 'is aws-related',
       'related service', 'keywords', 'point out deprecated version',
       'provide alternatives', 'share a link', 'provide code', 'others',
       'for question (1) / for answer (2)'],
      dtype='object')

In [47]:
data_answer.columns

Index(['Unnamed: 0', 'Id', 'PostId', 'link', 'outdated (manually checked)',
       'description', 'link outdated', 'answer outdated',
       'service in question outdated', 'other reasons', 'partly outdated',
       'totally outdated', 'is aws-related', 'related service', 'keywords',
       'phrases', 'strucutures', 'point out deprecated version',
       'provide alternatives', 'share a link', 'provide code', 'others',
       'Text'],
      dtype='object')

In [48]:
data_comment['is_comment'] = 1
data_answer['is_comment'] = 0

In [49]:
data = pd.concat([data_comment, data_answer], axis = 0, join = 'inner')

In [50]:
len(data)

258

In [51]:
data.columns

Index(['Id', 'PostId', 'Text', 'link', 'outdated (manually checked)',
       'link outdated', 'answer outdated', 'other reasons', 'partly outdated',
       'totally outdated', 'is aws-related', 'related service', 'keywords',
       'point out deprecated version', 'provide alternatives', 'share a link',
       'provide code', 'others', 'is_comment'],
      dtype='object')

## 3.1 BoW

In [341]:
#use only text as training data
X = data['Text']
y = data['outdated (manually checked)']

In [342]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [343]:
X_train = X_train.fillna('fillna')
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [55]:
len(X_train)

232

In [344]:
clf = MultinomialNB().fit(X_train_tfidf, y_train)
clfLR = LogisticRegression(C=50).fit(X_train_tfidf, y_train)

In [345]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [346]:
clf.score(X_test_tfidf, y_test)

0.7692307692307693

In [347]:
clfLR.score(X_test_tfidf, y_test)

0.6153846153846154

In [348]:
f1_score(y_test, clf.predict(X_test_tfidf))

0.8695652173913044

In [349]:
f1_score(y_test, clfLR.predict(X_test_tfidf))

0.761904761904762

## 3.2 Other Features

In [60]:
X = data[['link outdated',
       'answer outdated', 'other reasons', 'partly outdated',
       'totally outdated', 'is aws-related', 'keywords',
       'point out deprecated version', 'provide alternatives', 'share a link',
       'provide code', 'others', 'is_comment']]
y = data['outdated (manually checked)']

In [61]:
X.fillna('0')
key_word_dummies = pd.get_dummies(X.keywords, prefix='keyword_')
aws_related_dummies = pd.get_dummies(X['is aws-related'], prefix='aws_related_')
X = pd.concat([X, key_word_dummies], axis = 1)
X = pd.concat([X, aws_related_dummies], axis = 1)
X = X.drop(columns = 'keywords')
X = X.drop(columns = 'is aws-related')

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [63]:
X_train = X_train.fillna('0')
X_test = X_test.fillna('0')

In [64]:
clf = BernoulliNB().fit(X_train, y_train)

In [65]:
clf.score(X_test, y_test)

0.8653846153846154

In [66]:
f1_score(y_test, clf.predict(X_test))

0.9014084507042254

## 3.3 Predictions on Other Data (Answer)

In [135]:
answer_data = pd.read_csv('./data/stackoverflow_amazon_a_outdated.csv', encoding="ISO-8859-1")
answer_test_data = pd.read_csv('./data/AnswerTestData_1.csv', encoding="ISO-8859-1")

In [136]:
row_indices = []
for i in range(len(answer_data)):
    for j in range(len(answer_test_data)):
        if answer_data.iloc[i]['Id'] == answer_test_data.iloc[j]['Id']:
            row_indices.append(i)

In [137]:
answer_predict_data = answer_data.drop(row_indices)

In [138]:
len(answer_predict_data)

807

In [328]:
#use only text as training data
X = data['Text']
y = data['outdated (manually checked)']

predictions = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)
    X_train = X_train.fillna('fillna')
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    clf = LogisticRegression(C=50).fit(X_train_tfidf, y_train)

    X_test = answer_predict_data['Body']
    X_test_counts = count_vect.transform(X_test)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)

    prediction = clf.predict(X_test_tfidf)
    predictions.append(prediction)

In [329]:
counts = np.sum(predictions, axis = 0)

In [330]:
for i in range(len(counts)):
    if counts[i] == 5:
        prediction[i] = True
    else:
        prediction[i] = False

In [331]:
unique, counts = np.unique(prediction, return_counts=True)
dict(zip(unique, counts))

{False: 190, True: 617}

In [320]:
answer_test_data['outdated (manually checked)'].value_counts()

True     67
False    33
Name: outdated (manually checked), dtype: int64

In [321]:
data['outdated (manually checked)'].value_counts()

True     177
False     81
Name: outdated (manually checked), dtype: int64

In [219]:
answer_predict_data['prediction'] = prediction

In [220]:
answer_predict_data.head()

,Id,PostTypeId,ParentId,CreationDate,Score,Body,OwnerUserId,LastEditorUserId,LastEditDate,LastActivityDate,CommentCount,ContentLicense,OwnerDisplayName,CommunityOwnedDate,Unnamed: 14,link,prediction
30,9076551,2,9069447,2012-01-31T08:33:18.883,9,"<p>Given your constraints, the desired functio...",45773.0,-1.0,2017-05-23T11:53:17.133,2013-11-08T13:41:50.727,1,CC BY-SA 3.0,NaN,NaN,NaN,https://stackoverflow.com/questions/9076551,True
31,9179801,2,7487292,2012-02-07T16:20:03.357,4,"<p>Note that according to Amazon, at <a href=""...",98694.0,NaN,NaN,2012-02-07T16:20:03.357,0,CC BY-SA 3.0,NaN,NaN,NaN,https://stackoverflow.com/questions/9179801,True
32,9440295,2,9280539,2012-02-25T01:30:14.873,0,<p>I'd like to note that AWS is deprecated whe...,526849.0,NaN,NaN,2012-02-25T01:30:14.873,0,CC BY-SA 3.0,NaN,NaN,NaN,https://stackoverflow.com/questions/9440295,True
34,9560483,2,9559973,2012-03-05T01:06:55.170,1,<blockquote>\r <p>Is there a way to expand it...,45773.0,NaN,NaN,2012-03-05T01:06:55.170,3,CC BY-SA 3.0,NaN,NaN,NaN,https://stackoverflow.com/questions/9560483,False
35,9638670,2,9506791,2012-03-09T17:50:46.747,1,<p>i would say API documentation is outdated:<...,1246870.0,NaN,NaN,2012-03-09T17:50:46.747,2,CC BY-SA 3.0,NaN,NaN,NaN,https://stackoverflow.com/questions/9638670,True


In [221]:
answer_predict_data.to_csv('./data/answer_prediction_by_model.csv')

## 3.4 Predictions on Other Data (Comment)

In [267]:
answer_comment_data = pd.read_csv('./data/stackoverflow_amazon_a_comment_outdated.csv', encoding="ISO-8859-1")
question_comment_data = pd.read_csv('./data/stackoverflow_amazon_q_comment_outdated.csv', encoding="ISO-8859-1")
comment_test_data = pd.read_csv('./data/commentTestData.csv',error_bad_lines=False, sep='\t')

In [268]:
comment_data = pd.concat([answer_comment_data, question_comment_data], axis = 0)
comment_test_data = data_comment

In [254]:
comment_data.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId', 'link',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'ContentLicense'],
      dtype='object')

In [269]:
comment_data = comment_data.drop(columns = ['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'])
comment_predict_data = comment_data

In [270]:
row_indices = []
for i in range(len(comment_data)):
    for j in range(len(comment_test_data)):
        if comment_data.iloc[i]['Id'] == comment_test_data.iloc[j]['Id']:
            comment_predict_data = comment_predict_data.drop(i)
            row_indices.append(i)

KeyError: '[561] not found in axis'

In [265]:
comment_predict_data = comment_data.drop(917)

KeyError: '[917] not found in axis'